<a href="https://colab.research.google.com/github/david-j-cox/Man-vs-Machine/blob/mlb-matching/GME_fits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the packages we'll be using and read in the data set

In [7]:
# Packages we'll use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import zipfile, io, os
import math
import time
# from google.colab import files
# files.upload()
from google.colab import drive
drive.mount('/content/gdrive')
%cd './gdrive/My Drive/GME & MLB/Data_and_Fits'

/content/gdrive/My Drive/GME & MLB/Data_and_Fits


# Make sure dataframe is ready for fitting

In [16]:
# Read in file
data_raw = pd.read_csv("all_pitches_08_19.csv").drop(['Unnamed: 0'], axis=1)
# data_raw = data_raw.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
data = data_raw.copy()
data[::300000] # Take a look at every 300,000 row of the df

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,...,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,FF,2019-09-29,94.9,-2.3738,5.1379,Ian Kennedy,570731,453178,field_out,hit_into_play,NaN,NaN,NaN,NaN,3.0,Jonathan Schoop pops out to second baseman Eri...,R,R,R,KC,MIN,X,4.0,popup,1,1,2019,-0.909700,1.197200,0.3906,3.1900,NaN,NaN,NaN,2,9,Top,134.20,170.51,NaN,...,-16.6055,3.5400,1.6800,NaN,80.0,69.0,95.087,2430.0,6.245,565782,453178,664926.0,656811.0,625510.0,596144.0,600858.0,643436.0,621433.0,641531.0,54.2545,0.002,0.000,0.0,1.0,0.0,0.0,3.0,67,3,4-Seam Fastball,4,4,4,4,4,4,4,4,Standard,Standard
300000,FF,6/12/2013,92.7,NaN,NaN,Ubaldo Jimenez,595777,434622,single,hit_into_play_no_out,NaN,NaN,NaN,NaN,5.0,Jurickson Profar singles on a line drive to ri...,R,L,R,TEX,CLE,X,9.0,line_drive,3,2,2013,-1.000000,1.200000,0.2300,2.6600,NaN,NaN,NaN,0,3,Bot,170.68,120.48,NaN,...,-19.0500,3.2500,1.5000,NaN,NaN,NaN,NaN,NaN,NaN,347723,434622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9,NaN,1.0,0.0,NaN,25,6,4-Seam Fastball,1,3,1,3,3,1,1,3,NaN,NaN
600000,FF,8/5/2015,98.2,-2.2600,6.4000,Taijuan Walker,518934,592836,NaN,ball,NaN,NaN,NaN,NaN,11.0,NaN,R,R,R,COL,SEA,B,NaN,NaN,2,2,2015,-0.500000,0.900000,-1.0000,5.0600,NaN,NaN,NaN,2,6,Bot,NaN,NaN,NaN,...,-22.1420,3.7300,1.7400,NaN,NaN,NaN,98.800,2159.0,6.800,415263,592836,572287.0,444432.0,429664.0,572122.0,606466.0,429711.0,457706.0,443558.0,53.7400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52,5,4-Seam Fastball,3,3,3,3,3,3,3,3,Standard,Standard
900000,FF,2019-04-14,95.1,-2.4165,5.8449,Jacob Barnes,641355,606930,strikeout,called_strike,NaN,NaN,NaN,NaN,14.0,Cody Bellinger called out on strikes.,R,L,R,LAD,MIL,S,2.0,NaN,2,2,2019,-0.353100,1.198000,0.5948,1.4827,NaN,NaN,NaN,1,6,Bot,NaN,NaN,NaN,...,-14.9366,3.8740,1.7203,NaN,NaN,NaN,96.485,2332.0,6.791,565808,606930,518735.0,542583.0,519058.0,543768.0,606115.0,592325.0,456715.0,519346.0,53.7083,NaN,NaN,0.0,1.0,0.0,0.0,NaN,50,5,4-Seam Fastball,7,0,7,0,0,7,7,0,Infield shift,Standard
1200000,FF,2015-06-23,92.9,-0.6400,5.7700,Al Alburquerque,596019,456379,field_out,hit_into_play,NaN,NaN,NaN,NaN,1.0,Francisco Lindor lines out to center fielder A...,R,L,R,CLE,DET,X,8.0,line_drive,2,1,2015,-0.900000,1.500000,-0.7200,2.8500,570482.0,543401.0,NaN,2,7,Bot,113.87,93.28,NaN,...,-15.4010,3.3100,1.5300,279.0,101.5,13.0,91.100,2409.0,5.200,414715,456379,543510.0,408234.0,435079.0,592206.0,461865.0,493316.0,543238.0,434658.0,55.2400,0.733,0.764,0.0,1.0,0.0,0.0,4.0,62,4,4-Seam Fastball,1,7,1,7,7,1,1,7,Standard,Strategic
1500000,SL,7/24/2012,90.6,NaN,NaN,Clayton Kershaw,279577,477132,NaN,blocked_ball,NaN,NaN,NaN,NaN,60.0,NaN,R,R,L,STL,LAD,B,NaN,NaN,0,2,2012,-0.600000,0.900000,-1.0600,0.4400,501896.0,518614.0,425794.0,2,6,Bot,NaN,NaN,NaN,...,-23.7420,3.1800,1.4800,NaN,NaN,NaN,NaN,NaN,NaN,319183,477132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,3,Slider,4,2,4,2,2,4,4,2,NaN,NaN
1800000,SL,8/28/2013,76.7,NaN,NaN,Darren Oliver,407489,119984,NaN,called_strike,NaN,NaN,NaN,NaN,29.0,NaN,R,L,L,TOR,NYY,S,NaN,NaN,0,1,2013,-1.200000,-0.200000,0.6300,2.5700,NaN,NaN,NaN,1,9,Top,NaN,NaN,NaN,...,-35.4640,3.6200,1.6200,NaN,NaN,NaN,NaN,NaN,NaN,348729,119984,Na

In [13]:
# Create a list of all the unique pitchers in the df
uniq_p = data.player_name.unique()
uniq_p = uniq_p.tolist()
print("Number of unique pitchers in the dataset:", len(uniq_p))
print("Number of total observations in the dataset:", len(data))

Number of unique pitchers in the dataset: 2374
Number of total observations in the dataset: 8467473


In [17]:
# Look at all the different pitch types in the dataframe
data['pitch_name'].unique()

array(['4-Seam Fastball', 'Cutter', 'Knuckle Curve', 'Sinker', 'Changeup',
       'Slider', '2-Seam Fastball', 'Curveball', nan, 'Split Finger',
       'Intentional Ball', 'Split-Finger', 'Pitch Out', 'Fastball',
       'Unknown', 'Forkball', 'Knuckleball', 'Eephus', 'Knuckle Ball',
       'Screwball'], dtype=object)

In [18]:
# Create lists of which pitches fall into which categories
hrd_strt = ['4-Seam Fastball', '2-Seam Fastball', 'Cutter']
hrd_brkng = ['Slider', 'Sinker', 'Split Finger', 'Screwball', 'Forkball']
sft_strt = ['Changeup', 'Knuckle Ball', 'Eephus']
sft_brkng = ['Curveball', 'Knuckle Curve']

In [19]:
# Look at all the different outcome events
data['description'].unique()

array(['hit_into_play', 'ball', 'called_strike', 'swinging_strike',
       'foul', 'foul_tip', 'swinging_strike_blocked',
       'hit_into_play_no_out', 'blocked_ball', 'hit_by_pitch',
       'hit_into_play_score', 'foul_bunt', 'missed_bunt', 'bunt_foul_tip',
       'intent_ball', 'pitchout', 'swinging_pitchout',
       'pitchout_hit_into_play_no_out', 'pitchout_hit_into_play',
       'pitchout_hit_into_play_score', 'foul_pitchout', 'unknown_strike'],
      dtype=object)

In [20]:
# Create list of all Strike and out categories to serve as reinforcers
s_o = ['hit_into_play', 'foul','swinging_strike', 'swinging_strike_blocked', \
       'called_strike','foul_bunt', 'foul_tip', 'missed_bunt','swinging_pitchout', \
       'foul_pitchout', 'bunt_foul_tip', 'unknown_strike']

# Fit the GME for each pitcher in the df using strikes and outs as assumed reinforcers

### Caution: This will run for a long time. Break into batches if possible. 

In [ ]:
# Make sure data types are as we need them
data.game_year = data.game_year.astype(int)

player = []
season = []
VAC_strikes = []
bias_strikes = []
sens_strikes = []
player_errors = []

# Pull df for the target pitcher and break down by season
for i in uniq_p[1900:2000]: # out of 2374
  try:
    # Lists to store things in
    game_number = []
    year = []
    hard_straight_count = []
    hard_breaking_count = []
    soft_straight_count = []
    soft_breaking_count = []
    hard_straight_s_o = []
    hard_breaking_s_o = []
    soft_straight_s_o = []
    soft_breaking_s_o = []

    df = data[data['player_name']==i]  

    for j in [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]:
      try:
        df_year = df[df['game_year']==j]
        g_uniq = df_year['game_date'].unique()
        if len(g_uniq) < 5:
          continue
        else:
          gnum = 1
          for k in g_uniq:
            game_data = df_year[df_year['game_date']==k]

            if len(game_data) < 5:
              pass
            else:
              # Create dfs for each pitch type
              hard_straight = game_data.loc[game_data.pitch_name.isin(hrd_strt)]
              hard_breaking = game_data.loc[game_data.pitch_name.isin(hrd_brkng)]
              soft_straight = game_data.loc[game_data.pitch_name.isin(sft_strt)]
              soft_breaking = game_data.loc[game_data.pitch_name.isin(sft_brkng)]
              
              # Get the count of each pitch type per game
              hstrt = len(hard_straight)
              hbrk = len(hard_breaking)
              sstrt = len(soft_straight)
              sbrk = len(soft_breaking)

              # Get the count of strikes/outs that followed each pitch type per game
              hstrt_strike = len(hard_straight.loc[hard_straight.description.isin(s_o)])
              hbrk_strike = len(hard_breaking.loc[hard_breaking.description.isin(s_o)])
              sstrt_strike = len(soft_straight.loc[soft_straight.description.isin(s_o)])
              sbrk_strike = len(soft_breaking.loc[soft_breaking.description.isin(s_o)])
              
              # Append the vals to the respective lists
              game_number.append(gnum)
              gnum += 1
              year.append(j)
              # Pitches
              hard_straight_count.append(hstrt)
              hard_breaking_count.append(hbrk)
              soft_straight_count.append(sstrt)
              soft_breaking_count.append(sbrk)
              # Strikes/outs
              hard_straight_s_o.append(hstrt_strike)
              hard_breaking_s_o.append(hbrk_strike)
              soft_straight_s_o.append(sstrt_strike)
              soft_breaking_s_o.append(sbrk_strike)
          
          # Log pitch and outcome ratios using fastball as Bi
          logged_bx = []
          logged_strikes = []

          for l in list(range(len(hard_straight_count))):
            behavior = (hard_straight_count[l]+1)/(hard_breaking_count[l] + soft_straight_count[l] + soft_breaking_count[l]+1)
            strikes = (hard_straight_s_o[l]+1)/(hard_breaking_s_o[l] + soft_straight_s_o[l] + soft_breaking_s_o[l]+1)
            behavior = math.log(behavior, 10)
            strikes = math.log(strikes, 10)
            logged_bx.append(behavior)
            logged_strikes.append(strikes)
        
      except:
        continue

        # Combine the lists and save df for the pitcher
        count_data = pd.DataFrame({'Year':year, 'Game': game_number, \
                                  'HStrt': hard_straight_count, 'HBrk':hard_breaking_count, \
                                  'SStrt':soft_straight_count,'SBrk':soft_breaking_count, \
                                  'HStrtStrike':hard_straight_s_o, 'HBrkStrike':hard_breaking_s_o, \
                                  'SStrtStrike':soft_straight_s_o, 'SBrkStrike':soft_breaking_s_o, \
                                  'Logged_Bx':logged_bx, 'Logged_Strikes':logged_strikes})
        count_data = count_data.fillna(0)

        # Save the player data
        count_data.to_csv('%s_counts.csv' %i)

        # Fit the GME and return the parameters
        s_str, b_str = np.polyfit(count_data.Logged_Strikes, count_data.Logged_Bx, 1)
        corrmatSTR = np.corrcoef(count_data.Logged_Strikes, count_data.Logged_Bx)
        r2_str = (corrmatSTR[0,1])**2

        # Save the GME values
        player.append(i)
        season.append(j)
        VAC_strikes.append(round(r2_str, 4))
        bias_strikes.append(round(b_str, 4))
        sens_strikes.append(round(s_str, 4))

      # Save the data
      player_fits = pd.DataFrame({'Player':player, 'Season':season, \
                                  'VAC_SO':VAC_strikes, 'Bias_SO':bias_strikes, \
                                  'Sensitivity_SO':sens_strikes})
      player_fits.to_csv('All_GME_Fits_20.csv')
      print("Finished with year %s for %s" %(j, i))
    
  except Exception as ex:
    player_errors.append(i)
    print('Error with %s' %i)
    time.sleep(5)

Finished with year 2014 for Tyler Matzek
Finished with year 2015 for Tyler Matzek
Finished with year 2013 for Pedro Hernandez
Finished with year 2014 for Tsuyoshi Wada
Finished with year 2015 for Tsuyoshi Wada
Finished with year 2012 for A.J. Griffin
Finished with year 2013 for A.J. Griffin
Finished with year 2016 for A.J. Griffin
Finished with year 2017 for A.J. Griffin
Finished with year 2017 for Paolo Espino
Finished with year 2015 for Simon Castro
Finished with year 2017 for Simon Castro
Finished with year 2019 for Edgar Garcia
Finished with year 2019 for Cole Irvin
Finished with year 2019 for JD Hammer
Finished with year 2015 for Josh A. Smith
Finished with year 2016 for Josh A. Smith
Finished with year 2017 for Josh A. Smith
Finished with year 2019 for Josh A. Smith
Finished with year 2019 for Mike Shawaryn
Finished with year 2019 for Trevor Kelley
Finished with year 2019 for Travis Lakins
Finished with year 2019 for Josh Taylor
Finished with year 2019 for Darwinzon Hernandez
Fin

# Plot histogram of GME params and fits

In [ ]:
# IF PICKING UP FRESH
player_fits = pd.read_csv('/content/gdrive/My Drive/Matching with RE288/Data_and_Fits/ALL_GME_fits.csv')
player_fits = player_fits.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
player_fits.head()

In [ ]:
# Read in data on starting pitchers
starters = pd.read_csv('/content/gdrive/My Drive/Matching with RE288/Data_and_Fits/Starting_Pitchers_16_19.csv')
starters.head()

In [ ]:
# Create dfs then lists of starters for each year
start_2016 = starters[starters['Year']==2016]
start_2017 = starters[starters['Year']==2017]
start_2018 = starters[starters['Year']==2018]
start_2019 = starters[starters['Year']==2019]

starters_16_list = pd.DataFrame(start_2016.Name.unique())
starters_17_list = pd.DataFRame(start_2017.Name.unique())
starters_18_list = start_2018.Name.unique()
starters_19_list = start_2019.Name.unique()

# Print lengths
print('2016:', len(starters_16_list), "\n", '2017:', len(starters_17_list), "\n", \
      '2018:', len(starters_18_list), "\n", '2019:', len(starters_19_list), "\n", )

In [ ]:
starters_16_list

In [ ]:
df_2016 = player_fits[player_fits['Season']==2016] # create 2016 df
df_16_st = df_2016[df_2016['Player'].isin(starters_16_list)] # isolate starters and pitchers

In [ ]:
# Plot Vacs by year for starters and relievers
df_2016 = player_fits[player_fits['Season']==2016] # create 2016 df
df_16_st = df_2016[df_2016['Player'].isin(starters_16_list[0])] # isolate starters and pitchers
df_16_re = df_2016[~df_2016['Player'].isin(starters_16_list[0])]

plt.figure(figsize=(12, 8))
plt.hist(df_16_st.VAC_288, color='blue', alpha=0.8, bins=100, label='RE288-SR+; Starters')
plt.hist(df_16_st.VAC_SO, color='red', alpha=0.8, bins=100, label='Strikes-SR+; Starters')
plt.hist(df_16_re.VAC_288, color='darkviolet', alpha=0.8, bins=100, label='RE288-SR+; Relievers')
plt.hist(df_16_re.VAC_SO, color='gold', alpha=0.8, bins=100, label='Strikes-SR+; Relievers')
plt.legend(fontsize=16)
plt.yticks(fontsize=16)
plt.ylabel('Number of Pitchers', fontsize=24)
plt.xticks(fontsize=16)
plt.xlabel('VAC by GME', fontsize=24)
#plt.savefig('pop_VAC_hist_2.jpg')
plt.show()

In [ ]:
player_fits = player_fits.drop_duplicates(subset=['Player', 'VAC_288', 'VAC_SO', \
                                                    'Bias_288', 'Bias_SO', \
                                                    'Sensitivity_288', 'Sensitivity_SO'], \
                                          keep='last')
player_fits.to_csv('ALL_GME_fits.csv')

In [ ]:
plt.figure(figsize=(12, 8))
plt.hist(player_fits.VAC_288, color='blue', alpha=0.8, bins=100, label='RE288 Change as SR+')
plt.hist(player_fits.VAC_SO, color='red', alpha=0.8, bins=100, label='Strikes/Outs as SR+')
plt.legend(fontsize=16)
plt.yticks(fontsize=16)
plt.ylabel('Number of Pitchers', fontsize=24)
plt.xticks(fontsize=16)
plt.xlabel('VAC by GME', fontsize=24)
plt.savefig('pop_VAC_hist_2.jpg')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
plt.hist(player_fits.Bias_288, color='blue', alpha=0.8, bins=100, label='RE288 Change as SR+')
plt.hist(player_fits.Bias_SO, color='red', alpha=0.8, bins=100, label='Strikes/Outs as SR+')
plt.legend(fontsize=16)
plt.yticks(fontsize=16)
plt.ylabel('Number of Pitchers', fontsize=24)
plt.ylim(0, )
plt.xticks(fontsize=16)
plt.xlabel('Estimated Bias', fontsize=24)
plt.savefig('pop_bias_hist_2.jpg')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
plt.hist(player_fits.Sensitivity_288, color='blue', alpha=0.8, bins=100, label='RE288 Change as SR+')
plt.hist(player_fits.Sensitivity_SO, color='red', alpha=0.8, bins=100, label='Strikes/Outs as SR+')
plt.legend(fontsize=16)
plt.yticks(fontsize=16)
plt.ylabel('Number of Pitchers', fontsize=24)
plt.xticks(fontsize=16)
plt.xlabel('Estimated Sensitivity', fontsize=24)
plt.savefig('pop_Sensitivity_hist_2.jpg')
plt.show()

In [ ]:
# Fit GME for all pitchers for each season using RE288 as reinforcer. 

# Read in file
data_raw = pd.read_csv("All_MLB_data_for_GME.csv")
data_raw = data_raw.drop(['Unnamed: 0'], axis=1)
data = data_raw.copy()
data.head()

# Fill all NaNs for RE288_change col with 0s or rows where pitch description is unknown
data = data[data['description'].notna()]
data = data[data['pitch_name'].notna()]
data['RE288_change'] = data['RE288_change'].fillna(0)
data = data.reset_index()

#uniq_p = uniq_p.tolist()
uniq_p.index('Edubray Ramos')

data.game_date.min()

uniq_p = uniq_p.tolist()

# Create a list of all the unique pitchers in the df
uniq_p = data.player_name.unique()
uniq_p = uniq_p.tolist()
print(len(uniq_p))

# Remove pitchers who have been troublesome
all_removed = ['Ryan Buchter', 'Tyson Ross', 'Ariel Pena', 'Dan Winkler', 'Andrew Heaney', \
                'Logan Kensing', 'Arnold Leon', 'Kyle Drabek', 'Chris Heston', \
                'Charlie Morton', 'John Danks', 'Robbie Erlin', 'Glen Perkins', \
                'Matt Buschmann', 'Tim Melville', 'Tyler Olson', 'David Hale', \
                'Tyler Wagner', 'Mike Broadway', 'Branden Pinder', 'Matt Marksberry', \
                'Erik Kratz', 'Damien Magnifico', 'Michael Kirkman', 'Daniel Webb', \
                'Vin Mazzaro', 'J.R. Graham', 'Scott Carroll', 'Josh Phegley', \
                'Layne Somsen', 'Kendry Flores', 'Alec Mills', 'Conor Mullee', \
                'Dustin Antolin', 'Tayron Guerrero', 'Chin-hui Tsao', 'Ruben Tejada', \
                'Ryan Merritt', 'Christian Bethancourt', 'Alexi Amarista', 'Cody Hall', \
                'Tyler Ladendorf', 'David Huff', 'JB Shuck', 'Tyler Danish', 'Al Alburquerque', \
                'Scott Diamond', 'Andrew Romine', 'Adrian Sampson', 'Jorge Rondon', \
                'Curtis Partch', 'Jonathan Aro', 'Gerardo Concepcion', 'Nick Tepesch', \
                'Donn Roach', 'Drew Butera', 'Yohan Flande', 'Chi Chi Gonzalez', \
                'Jo-Jo Reyes', 'Darwin Barney', 'Ryan Goins', 'Taylor Motter', \
                'Shawn Morimando', 'Miguel Montero', 'Chris Gimenez', 'Jayson Aquino', \
                'Dustin Molleken', 'Michael Roth', 'TJ House', 'Jared Hoying', \
                'Carlos Frias', 'Hyun-Jin Ryu', 'Kyle Lohse', 'Mike Mayers', \
                'Luis Sardinas', 'Roberto Hernandez', 'Eduardo Escobar', \
                'Joe Wieland', 'Jason Hursh', 'Brett Anderson', 'Tyler White', \
                'Ryan Flaherty', 'Adalberto Mejia', 'Parker Bridwell', 'Tyler Holt', \
                'Vicente Campos', 'Jake Esch', 'Kelvin Marte', 'Ben Rowen', \
                'Jose De Leon', 'Jake Buchanan', 'Ty Blach', 'Yohander Mendez', \
                'Jake Smith', 'David Paulino', 'Jason Vargas']
#uniq_p.remove(all_removed)
uniq_p = [i for i in uniq_p if i not in all_removed]
len(uniq_p)

# Identify which pitches fall in which categories
hrd_strt = ['4-Seam Fastball', '2-Seam Fastball', 'Cutter']
hrd_brkng = ['Slider', 'Sinker', 'Split Finger', 'Screwball', 'Forkball']
sft_strt = ['Changeup', 'Knuckle Ball', 'Eephus']
sft_brkng = ['Curveball', 'Knuckle Curve']

# Strike and out categories 
s_o = ['hit_into_play', 'foul','swinging_strike', 'swinging_strike_blocked', \
       'called_strike','foul_bunt', 'foul_tip', 'missed_bunt','swinging_pitchout', \
       'pitchout_hit_into_play_score','foul_pitchout', 'bunt_foul_tip']

# Calculate GME for each pitcher in df using RE288 as outcome
# Make sure data types are as we need them
data.game_year = data.game_year.astype(int)

player = []
season = []
VAC_RE288 = []
bias_RE288 = []
sens_RE288 = []
VAC_strikes = []
bias_strikes = []
sens_strikes = []
player_errors = []

# Pull df for the target pitcher and break down by season
for i in uniq_p[700:800]: # out of 1359
  try:
    # Lists to store things in
    game_number = []
    year = []
    hard_straight_count = []
    hard_breaking_count = []
    soft_straight_count = []
    soft_breaking_count = []
    hard_straight_re288 = []
    hard_breaking_re288 = []
    soft_straight_re288 = []
    soft_breaking_re288 = []
    hard_straight_s_o = []
    hard_breaking_s_o = []
    soft_straight_s_o = []
    soft_breaking_s_o = []

    df = data[data['player_name']==i]  

    for j in [2016, 2017, 2018, 2019]:
      df_year = df[df['game_year']==j]
      g_uniq = df_year['game_date'].unique()
      if len(g_uniq) < 5:
        pass
      else:
        gnum = 1
        for k in g_uniq:
          game_data = df_year[df_year['game_date']==k]

          if len(game_data) < 5:
            pass
          else:
            # Create dfs for each pitch type
            hard_straight = game_data.loc[game_data.pitch_name.isin(hrd_strt)]
            hard_breaking = game_data.loc[game_data.pitch_name.isin(hrd_brkng)]
            soft_straight = game_data.loc[game_data.pitch_name.isin(sft_strt)]
            soft_breaking = game_data.loc[game_data.pitch_name.isin(sft_brkng)]
            
            # Get the count of each pitch type per game
            hstrt = len(hard_straight)
            hbrk = len(hard_breaking)
            sstrt = len(soft_straight)
            sbrk = len(soft_breaking)
            
            # Get the sum of RE288 by pitch type for each game
            hstrt288 = hard_straight.loc[hard_straight['RE288_change']>0]
            hbrk288  = hard_breaking.loc[hard_breaking['RE288_change']>0]
            sstrt288 = soft_straight.loc[soft_straight['RE288_change']>0]
            sbrk288  = soft_breaking.loc[soft_breaking['RE288_change']>0]
            hstrt288 = hstrt288['RE288_change'].sum()
            hbrk288  = hbrk288['RE288_change'].sum()      
            sstrt288 = sstrt288['RE288_change'].sum()
            sbrk288  = sbrk288['RE288_change'].sum()

            # Get the count of strikes/outs that followed each pitch type per game
            hstrt_strike = len(hard_straight.loc[hard_straight.description.isin(s_o)])
            hbrk_strike = len(hard_breaking.loc[hard_breaking.description.isin(s_o)])
            sstrt_strike = len(soft_straight.loc[soft_straight.description.isin(s_o)])
            sbrk_strike = len(soft_breaking.loc[soft_breaking.description.isin(s_o)])
            
            # Append the vals to the respective lists
            game_number.append(gnum)
            gnum += 1
            year.append(j)
            # Pitches
            hard_straight_count.append(hstrt)
            hard_breaking_count.append(hbrk)
            soft_straight_count.append(sstrt)
            soft_breaking_count.append(sbrk)
            # RE288
            hard_straight_re288.append(hstrt288)
            hard_breaking_re288.append(hbrk288)
            soft_straight_re288.append(sstrt288)
            soft_breaking_re288.append(sbrk288)
            # Strikes/outs
            hard_straight_s_o.append(hstrt_strike)
            hard_breaking_s_o.append(hbrk_strike)
            soft_straight_s_o.append(sstrt_strike)
            soft_breaking_s_o.append(sbrk_strike)
          
        # Log pitch and outcome ratios using fastball as Bi
        logged_bx = []
        logged_RE288 = []
        logged_strikes = []

        for l in list(range(len(hard_straight_count))):
          behavior = (hard_straight_count[l]+1)/(hard_breaking_count[l] + soft_straight_count[l] + soft_breaking_count[l]+1)
          RE288 = (hard_straight_re288[l]+.0001)/(hard_breaking_re288[l] + soft_straight_re288[l] + soft_breaking_re288[l]+.0001)
          strikes = (hard_straight_s_o[l]+1)/(hard_breaking_s_o[l] + soft_straight_s_o[l] + soft_breaking_s_o[l]+1)
          behavior = math.log(behavior, 10)
          RE288 = math.log(RE288, 10)
          strikes = math.log(strikes, 10)
          logged_bx.append(behavior)
          logged_RE288.append(RE288)
          logged_strikes.append(strikes)
      
      # Combine the lists and save df for the pitcher
      count_data = pd.DataFrame({'Year':year, 'Game': game_number, \
                                'HStrt': hard_straight_count, 'HBrk':hard_breaking_count, \
                                'SStrt':soft_straight_count,'SBrk':soft_breaking_count, \
                                'HStrtRE288':hard_straight_re288, 'HBrkRE288':hard_breaking_re288, \
                                'SStrtRE288':soft_straight_re288, 'SBrkRE288':soft_straight_re288, \
                                'HStrtStrike':hard_straight_s_o, 'HBrkStrike':hard_breaking_s_o, \
                                'SStrtStrike':soft_straight_s_o, 'SBrkStrike':soft_breaking_s_o, \
                                'Logged_Bx':logged_bx, 'Logged_RE288':logged_RE288, \
                                'Logged_Strikes':logged_strikes})
      count_data = count_data.fillna(0)

      # Save the player data
      count_data.to_csv('%s_counts.csv' %i)

      # Fit the GME and return the parameters
      s_288, b_288 = np.polyfit(count_data.Logged_RE288, count_data.Logged_Bx, 1)
      corrmat288 = np.corrcoef(count_data.Logged_RE288, count_data.Logged_Bx)
      r2_288 = (corrmat288[0,1])**2
      s_str, b_str = np.polyfit(count_data.Logged_Strikes, count_data.Logged_Bx, 1)
      corrmatSTR = np.corrcoef(count_data.Logged_Strikes, count_data.Logged_Bx)
      r2_str = (corrmatSTR[0,1])**2

      # Save the GME values
      player.append(i)
      season.append(j)
      VAC_RE288.append(round(r2_288, 4))
      VAC_strikes.append(round(r2_str, 4))
      bias_RE288.append(round(b_288, 4))
      sens_RE288.append(round(s_288, 4))
      bias_strikes.append(round(b_str, 4))
      sens_strikes.append(round(s_str, 4))

      # Plot RE288
      min_val = (count_data[['Logged_RE288', 'Logged_Bx']].min().min()) - .1
      max_val = (count_data[['Logged_RE288', 'Logged_Bx']].max().max()) + .1
      plt.figure(figsize=(8, 8))
      plt.plot(count_data.Logged_RE288, count_data.Logged_Bx, 'o', color='blue', alpha=0.5, label='RE288')
      plt.plot(count_data.Logged_RE288, s_288*count_data.Logged_RE288 + b_288, color='blue')
      plt.yticks(fontsize=14)
      plt.ylabel('log(Bi/Bo)', fontsize=24)
      plt.ylim(min_val, max_val)
      plt.xticks(fontsize=14)
      plt.xlabel('log(RE288-Bi/RE288-Bo)', fontsize=24)
      plt.xlim(min_val, max_val)
      plt.title('%s RE288' %i, fontsize=24)
      plt.text(min_val+0.05, max_val-0.05, 'bias=%s\nsensitivity=%s\nVAC=%s' %(round(b_288, 2), round(s_288, 2), round(r2_288, 2)), \
              fontsize=16, horizontalalignment='left', verticalalignment='top')
      plt.savefig('%s GME_288.jpg' %i)
      plt.show()

      # Plot Strikes
      min_val = (count_data[['Logged_Strikes', 'Logged_Bx']].min().min()) - .1
      max_val = (count_data[['Logged_Strikes', 'Logged_Bx']].max().max()) + .1
      plt.figure(figsize=(8, 8))
      plt.plot(count_data.Logged_Strikes, count_data.Logged_Bx, 'o', color='red', alpha=0.5, label='RE288')
      plt.plot(count_data.Logged_Strikes, s_str*count_data.Logged_Strikes + b_str, color='red')
      plt.yticks(fontsize=14)
      plt.ylabel('log(Bi/Bo)', fontsize=24)
      plt.ylim(min_val, max_val)
      plt.xticks(fontsize=14)
      plt.xlabel('log(Strikes-Bi/Strikes-Bo)', fontsize=24)
      plt.xlim(min_val, max_val)
      plt.title('%s Strikes/Outs' %i, fontsize=24)
      plt.text(min_val+0.05, max_val-0.05, 'bias=%s\nsensitivity=%s\nVAC=%s' %(round(b_str, 2), round(s_str, 2), round(r2_str, 2)), \
              fontsize=16, horizontalalignment='left', verticalalignment='top')
      plt.savefig('%s GME_SO.jpg' %i)
      plt.show()

    # Save the data
    player_fits = pd.DataFrame({'Player':player, 'Season':season, \
                                'VAC_288':VAC_RE288, 'VAC_SO':VAC_strikes, \
                                'Bias_288':bias_RE288, 'Bias_SO':bias_strikes, \
                                'Sensitivity_288':sens_RE288, 'Sensitivity_SO':sens_strikes})
    player_fits.to_csv('All_GME_Fits_08.csv')
  
  except Exception as ex:
    player_errors.append(i)
    print('Error with %s' %i)
    time.sleep(5)

